# nn.ConvTranspose2d
## 1. nn.ConvTranspose2d 的作用

- 正向卷积 (`nn.Conv2d`) 会把输入特征图做卷积运算，通常用于 `下采样`/`提取特征`。

- 转置卷积 (nn.ConvTranspose2d) 是正向卷积的“逆操作”，可以把输入 `上采样`（`扩大空间尺寸`），常用于：

    - 图像生成（GAN）
    
    - 语义分割（UNet）
    
    - 自编码器（Decoder部分）

它并不是严格的“反卷积”，而是通过 `转置卷积核`的运算 来实现特征图的放大。

## 2. 参数详解

In [ ]:
torch.nn.ConvTranspose2d(
    in_channels,     # 输入通道数
    out_channels,    # 输出通道数
    kernel_size,     # 卷积核大小
    stride=1,        # 步幅
    padding=0,       # 填充
    output_padding=0,# 输出填充，解决上采样尺寸不对齐问题
    groups=1,        # 分组卷积
    bias=True,       # 是否带偏置
    dilation=1       # 卷积核元素之间的间隔
)


### 关键参数说明：

1、in_channels / out_channels

- 和普通卷积一样，`输入通道数`、`输出通道数`。

- 比如输入 `[N, 64, 16, 16]`，如果 `out_channels=32`，输出就是 `[N, 32, H_out, W_out]`。

2、kernel_size

- 卷积核大小，可以是单个数字或二元组，如 3 或 (3,3)。

- 控制每次上采样的感受野。

3、stride

- 控制输出特征图的放大倍数。

- 比如 `stride=2`，输入 `16x16` → 输出约 `32x32`。

4、padding

- 对输入进行零填充（影响输出大小）。

- 在转置卷积中，它和 `stride`、`kernel_size` 一起决定最终输出尺寸。

5、output_padding

- 用来 `精确控制输出尺寸`，尤其当目标尺寸不是 `stride` 的整数倍时。

- 例如 `stride`=2 但想得到 `33x33` 而不是` 32x32 `时，可以设置` output_padding=1`。

6、dilation

- 控制卷积核的膨胀程度，一般保持 1 即可。

---
## 3. 输出尺寸计算公式

设输入特征图大小为 `H_in × W_in`，输出为 `H_out × W_out`：

𝐻𝑜𝑢𝑡=(𝐻𝑖𝑛−1)×𝑠𝑡𝑟𝑖𝑑𝑒−2×𝑝𝑎𝑑𝑑𝑖𝑛𝑔+𝑑𝑖𝑙𝑎𝑡𝑖𝑜𝑛×(𝑘𝑒𝑟𝑛𝑒𝑙_𝑠𝑖𝑧𝑒−1)+𝑜𝑢𝑡𝑝𝑢𝑡_𝑝𝑎𝑑𝑑𝑖𝑛𝑔+1

𝑊𝑜𝑢𝑡=(𝑊𝑖𝑛−1)×𝑠𝑡𝑟𝑖𝑑𝑒−2×𝑝𝑎𝑑𝑑𝑖𝑛𝑔+𝑑𝑖𝑙𝑎𝑡𝑖𝑜𝑛×(𝑘𝑒𝑟𝑛𝑒𝑙_𝑠𝑖𝑧𝑒−1)+𝑜𝑢𝑡𝑝𝑢𝑡_𝑝𝑎𝑑𝑑𝑖𝑛𝑔+1

---
## 4. 使用示例
基本用法：

In [ ]:
import torch
import torch.nn as nn

# 输入 [N, 16, 8, 8] -> 输出 [N, 33, 15, 15]
conv_trans = nn.ConvTranspose2d(
    in_channels=16, 
    out_channels=33, 
    kernel_size=3, 
    stride=2, 
    padding=1, 
    output_padding=1
)

x = torch.randn(20, 16, 8, 8)
y = conv_trans(x)

print(y.shape)  # torch.Size([20, 33, 15, 15])


In [ ]:
在网络中使用（例如 GAN 生成器）：

In [ ]:
import torch
import torch.nn as nn

# 定义生成器类，继承自 nn.Module
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3):
        # 调用父类构造函数
        super(Generator, self).__init__()
        
        # 定义生成器的主干网络（使用 Sequential 容器组织层）
        self.net = nn.Sequential(
            # 第一层：转置卷积（上采样），将 1x1 的噪声向量扩展为 4x4 特征图
            # 输入：[batch_size, z_dim, 1, 1] → 输出：[batch_size, 128, 4, 4]
            # kernel_size=4, stride=1, padding=0，从 1x1 → 4x4
            nn.ConvTranspose2d(z_dim, 128, 4, 1, 0, bias=False),
            nn.BatchNorm2d(128),     # 批归一化，稳定训练
            nn.ReLU(True),           # 激活函数，引入非线性

            # 第二层：转置卷积，将 4x4 → 8x8
            # 输入：[batch_size, 128, 4, 4] → 输出：[batch_size, 64, 8, 8]
            # kernel_size=4, stride=2, padding=1，实现 2 倍上采样
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            # 第三层：转置卷积，将 8x8 → 16x16，输出图像通道数（如 RGB 为 3）
            # 输入：[batch_size, 64, 8, 8] → 输出：[batch_size, img_channels, 16, 16]
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()                # 输出范围 [-1, 1]，适合图像生成（通常输入图像已归一化至此范围）
        )

    # 前向传播函数
    def forward(self, x):
        return self.net(x)           # 将输入 x 通过定义好的网络结构


# 创建一个 batch_size=16 的随机噪声向量 z
# 形状为 [16, 100, 1, 1]：16 个样本，每个样本是 100 维向量，以 1x1 空间维度输入（符合第一层 ConvTranspose2d 输入要求）
z = torch.randn(16, 100, 1, 1)

# 实例化生成器模型
gen = Generator()

# 将噪声输入生成器，生成图像
img = gen(z)

# 打印输出图像的形状
print(img.shape)  # 输出：torch.Size([16, 3, 16, 16])
# 表示生成了 16 张 3 通道（如 RGB）、16x16 分辨率的图像

## 5. 小结

- `nn.ConvTranspose2d` 主要用于 `上采样`，常见于`生成模型`和`解码器`。

- 核心参数：`stride`（决定放大倍数）、`padding`、`output_padding`（修正输出大小）。

- `输出尺寸公式要记住`，特别在搭建网络时避免尺寸对不上。